In [16]:
# CREATE CASSANDRA CONNECTION
import cassandra
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [17]:
# CREATE A KEYSPACE TO DO OUR WORK
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

# SET THE KEYSPACE/SCHEMA NAME
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

Imagine we would like to start creating a new Music Library of albums.

- We want to ask 1 question of our data
    -  Give me every album in my music library that was released by an Artist with Albumn Name in `DESC` Order and City In `DESC` Order
    - select * from music_library WHERE ARTIST_NAME="The Beatles"
    
- How should we model this data? What should be our Primary Key and Partition Key? 
- Since our data is looking for the ARTIST_NAME let's start with that. From there we will need to add other elements to make sure the Key is unique. We also need to add the CITY and ALBUM_NAME as Clustering Columns to sort the data. That should be enough to make the row key unique 

Table Name: music_library

    - column 1: Year
    - column 2: Artist Name
    - column 3: Album Name
    - Column 4: City
    - PRIMARY KEY(artist name, album name, city)

In [18]:
query = "CREATE TABLE IF NOT EXISTS music_library " 
query = query + "(year int, artist_name text, album_name text, city text, PRIMARY KEY (artist_name, album_name, city))"
try:
 session.execute(query)
except Exception as e:
 print(e)

In [19]:
query = "INSERT INTO music_library (year, artist_name, album_name, city)"
query = query + " VALUES (%s, %s, %s, %s)"

try:
 session.execute(query, (1970, "The Beatles", "Let it Be", "Liverpool"))
except Exception as e:
 print(e)

try:
 session.execute(query, (1965, "The Beatles", "Rubber Soul", "Oxford"))
except Exception as e:
 print(e)

try:
 session.execute(query, (1964, "The Beatles", "Beatles For Sale", "London"))
except Exception as e:
 print(e)

try:
 session.execute(query, (1966, "The Monkees", "The Monkees", "Los Angeles"))
except Exception as e:
 print(e)

try:
 session.execute(query, (1970, "The Carpenters", "Close To You", "San Diego"))
except Exception as e:
 print(e)

In [20]:
# LET'S VALIDATE OUR DATA MODEL
query = "select * from music_library WHERE ARTIST_NAME='The Beatles' ALLOW FILTERING"
try:
 rows = session.execute(query)
except Exception as e:
 print(e)

for row in rows:
 print (row.artist_name, row.album_name, row.city, row.year)

The Beatles Beatles For Sale London 1964
The Beatles Let it Be Liverpool 1970
The Beatles Rubber Soul Oxford 1965


In [21]:
# FOR DEMO PURPOSES DROP THE TABLE
try:
 session.execute('Drop table music_library')
except Exception as e:
 print(e)

In [22]:
# CLOSE THE SESSION AND CLUSTER CONNECTION
session.shutdown()
cluster.shutdown()